<a href="https://colab.research.google.com/github/Herais/NLP_Learning_by_Selective_Data/blob/main/Open.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Navigate to project directory
import os
if not os.path.exists('/content/drive/MyDrive/Github'):
  !mkdir Github
path_github   = '/content/drive/MyDrive/Github'
os.chdir(path_github)

# git clone
#!git clone https://github.com/Herais/NLP_Learning_by_Selective_Data.git
path_wd = path_github + '/' + 'NLP_Learning_by_Selective_Data'
os.chdir(path_wd)
!pwd

In [ ]:
# INSTALL REQUIRED PACKAGES
!pip install -r requirements.txt
%tensorflow_version 1.x

In [ ]:
# LOAD AND EXPLORE DATASETS

# Import Libaries
import numpy as np
import pandas as pd

# Path
path_datasets ='/datasets'

# Load Dataset: Movie Title and Summary
path_dataset_movie = '/Movie_Title_and_Summary'
df_movie = pd.read_json(path_wd + path_datasets + path_dataset_movie + '/train.json', orient='records', lines=True)
df_movie.head(10)

# Load Dataset: Computer Research Literature (CSL) Title and Abstract
path_dataset_csl = '/CSL'
df_csl = pd.read_json(path_wd + path_datasets + path_dataset_csl + '/train.json', orient='records', lines=True)
df_csl.head(10)

In [ ]:
# SET MODEL PARAMETERS

# Set Model Parameter
epochs = 10
batch_size = 8
maxlen = 256
topk = 1

# Download pretrained BERT model if necesary
path_pretrained_model = '/pretrained_model'
path_bert_chinese = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12'

if not os.path.exists(path_bert_chinese):
 # Download Model
  !wget https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip -P './pretrained_model/'
  %cd pretrained_model
  !unzip chinese_L-12_H-768_A-12.zip
  !rm -f chinese_L-12_H-768_A-12.zip
  os.chdir(path_wd)
  !pwd

# Set Pretrained Model Parameters (BERT)
config_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = path_wd + path_pretrained_model + '/chinese_L-12_H-768_A-12/vocab.txt'


# Load Vocab
from bert4keras.tokenizers import Tokenizer, load_vocab

token_dict, keep_tokens = load_vocab(
    dict_path=dict_path,
    simplified=True,
    startswith=['[PAD]', '[UNK]', '[CLS]', '[SEP]'],
)
tokenizer = Tokenizer(token_dict, do_lower_case=True)



Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
# TRAIN MODELS
# Skip to LOAD TRAINED MODELS to use existing models for prediction

In [ ]:
# LOAD TRAINED MODELS
from tensorflow import keras
from bert4keras.models import build_transformer_model

path_models = '/models'

# the Star Chaser Model trained with movie dataset only
model_StarChaser = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,  # include only tokens in keep tokens
)
model_StarChaser.load_weights(path_wd + path_models + '/best_model_StarChaser.weights')

# the Scholar Model trained with Computer Science Literature only
model_Scholar = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
)
model_Scholar.load_weights(path_wd + path_models + '/best_model_Scholar.weights')

# the Know-it-all Model Trained with both movie and cscl dataset
model_Knowitall = build_transformer_model(
    config_path,
    checkpoint_path,
    application='unilm',
    keep_tokens=keep_tokens,   # include only tokens in keep tokens
) 
model_Knowitall.load_weights(path_wd + path_models + '/best_model_Knowitall.weights')

In [ ]:
import utils
utils.save_model_with_dt(path_wd + path_models, model_Scholar, 'save_test')


In [ ]:
# PREDICTION
model_Scholar.

[None, None]

In [ ]:
# RESULT ANALYSIS